In [1]:
import re

import numpy as np
import pandas as pd
import openpyxl
import warnings
warnings.filterwarnings('ignore')

import unicodedata

In [2]:
%cd C:\Users\Administrador.PUESTO09\Desktop\Dimal_app_ clientes\data\Sucia

C:\Users\Administrador.PUESTO09\Desktop\Dimal_app_ clientes\data\Sucia


In [4]:
df_raw = pd.read_excel('../Sucia/Tareas.xlsm', header=None, engine='openpyxl')
print(df_raw.head(20))


                                                    0
0                                       PROCEDIMIENTO
1                                                 NaN
2                                                 NaN
3                                                 NaN
4          1. Armar archivor Rutas.xlsm y Tareas.xlsm
5                                                 NaN
6   Ir a Make, escenario Rutas - Lista tareas todo...
7               Ejecutar escenario, en filtros poner:
8                                                 NaN
9                                                 NaN
10                                                NaN
11                      Rutas de todos los vendedores
12                                                NaN
13                                 Visitas Periodicas
14                                                NaN
15        Clientes Especiales y Clientes Especiales 2
16                                                NaN
17                          

In [15]:
import openpyxl

wb = openpyxl.load_workbook('../Sucia/Tareas.xlsm', data_only=True)
print(wb.sheetnames)


['PROC', 'Tareas', 'Comercios Tareas', 'Otros a agregar', 'Tareas Todoist']


### Tareas.xlsx -> Tareas-limpio.xlsx(Proyectos)

In [16]:

try:
    df_tareas = pd.read_excel(
        '../Sucia/Tareas.xlsm',
        sheet_name='Tareas',
        engine='openpyxl'
    )
    print("Archivo cargado exitosamente")
    print(f"- Filas iniciales: {df_tareas.shape[0]}")
    print(f"- Columnas iniciales: {df_tareas.shape[1]}")
    print("\nColumnas disponibles:")
    for col in df_tareas.columns:
        print(f"- {col}")

except Exception as e:
    print(f"❌ Error al cargar archivo: {e}")
    raise



Archivo cargado exitosamente
- Filas iniciales: 9998
- Columnas iniciales: 16

Columnas disponibles:
- PROYECTO
- TAREA
- DESCRIPCION
- SECCION
- FECHA
- PERIODICIDAD
- ORDEN
- RESPONSABLE
- CODPROY
- CODTAREA
- ETIQUETA
- CODIGO
- TIPO
- ORDEN.1
- CODCLI2
- Unnamed: 15


In [17]:
# 2- Dejo solo las columnas que necesito: 

df_tareas = df_tareas[["PROYECTO", "TAREA", "CODIGO","PERIODICIDAD", "ORDEN", "TIPO"]]
df_tareas.sample(10)

,PROYECTO,TAREA,CODIGO,PERIODICIDAD,ORDEN,TIPO
4743,NaN,NaN,NaN,NaN,NaN,NaN
2579,NaN,NaN,NaN,NaN,NaN,NaN
7868,NaN,NaN,NaN,NaN,NaN,NaN
8968,NaN,NaN,NaN,NaN,NaN,NaN
9265,NaN,NaN,NaN,NaN,NaN,NaN
3674,NaN,NaN,NaN,NaN,NaN,NaN
3602,NaN,NaN,NaN,NaN,NaN,NaN
1509,Clientes Especiales,"22336, Ancap San Luis:",22336,NaN,140,Clie
3737,NaN,NaN,NaN,NaN,NaN,NaN
3109,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# 3. Elimino códigos nulos. 

df_tareas = df_tareas.dropna(subset=['CODIGO'])



In [19]:
# 4.  Un poco de limpieza: 

# Función para limpiar texto

def strip_accents(text):
  if pd.isna(text):
      return text
  return ''.join(c for c in unicodedata.normalize('NFD', str(text))
                if unicodedata.category(c) != 'Mn')

# Aplicar limpieza a PROYECTO
df_tareas['PROYECTO'] = df_tareas['PROYECTO'].apply(strip_accents)
print("✓ Acentos removidos de la columna PROYECTO")

✓ Acentos removidos de la columna PROYECTO


In [21]:
df_tareas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1724 entries, 9 to 1749
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PROYECTO      1724 non-null   object
 1   TAREA         1724 non-null   object
 2   CODIGO        1724 non-null   object
 3   PERIODICIDAD  1459 non-null   object
 4   ORDEN         1724 non-null   object
 5   TIPO          1724 non-null   object
dtypes: object(6)
memory usage: 94.3+ KB


In [22]:
df_tareas.head(5)

,PROYECTO,TAREA,CODIGO,PERIODICIDAD,ORDEN,TIPO
9,Ruta Dario Lunes,"11975, DUNAS,PARQUE DEL PLATA NORTE ,3%dto",11975,cada lunes,1,Ruta
10,Ruta Dario Lunes,"11629, MOUNTLINE S.A. (pasa pedido y se entreg...",11629,cada lun 9:00,2,Ruta
11,Ruta Dario Lunes,"62783,ANDREA ESCOBAR,( pasar a cobrar)",62783,cada lun 9:00,3,Ruta
12,Ruta Dario Lunes,"80233,MINIMARKET Y ROTI EL REPECHO, San Luis,",80233,cada lunes,4,Ruta
13,Ruta Dario Lunes,"80203, KIOSCO LO NUESTRO (Poner ubicacion en c...",80203,cada vie,5,Ruta


In [23]:
# Tipo de datos 

# Pasar columnas a categóricas
cols_categoricas = ['PROYECTO', 'PERIODICIDAD', 'TIPO']
for col in cols_categoricas:
    df_tareas[col] = df_tareas[col].astype('category')

# Pasar columnas a numéricas (forzando errores a NaN)
cols_numericas = ['CODIGO', 'ORDEN']
for col in cols_numericas:
    df_tareas[col] = pd.to_numeric(df_tareas[col], errors='coerce')

# Revisar resultado
print(df_tareas.dtypes)

PROYECTO        category
TAREA             object
CODIGO           float64
PERIODICIDAD    category
ORDEN            float64
TIPO            category
dtype: object


In [26]:
df_tareas.head()

,PROYECTO,TAREA,CODIGO,PERIODICIDAD,ORDEN,TIPO
9,Ruta Dario Lunes,"11975, DUNAS,PARQUE DEL PLATA NORTE ,3%dto",11975.0,cada lunes,1.0,Ruta
10,Ruta Dario Lunes,"11629, MOUNTLINE S.A. (pasa pedido y se entreg...",11629.0,cada lun 9:00,2.0,Ruta
11,Ruta Dario Lunes,"62783,ANDREA ESCOBAR,( pasar a cobrar)",62783.0,cada lun 9:00,3.0,Ruta
12,Ruta Dario Lunes,"80233,MINIMARKET Y ROTI EL REPECHO, San Luis,",80233.0,cada lunes,4.0,Ruta
13,Ruta Dario Lunes,"80203, KIOSCO LO NUESTRO (Poner ubicacion en c...",80203.0,cada vie,5.0,Ruta


In [31]:
# Algunas transformaciones: 

#Código a entero ( asú no trae decimales. 

df_tareas['CODIGO'] = df_tareas['CODIGO'].astype('Int64')
# Columna tarea que es el nombre de cliente lo voy a dejar ya que no voy a asociar proyecto a cliente predictivo. 
df_tareas.rename(columns={"TAREA": "TAREACLIENTE"}, inplace=True)

In [32]:
# 6. Guardado de archivo

df_tareas.to_excel("../Limpia/Tareas-limpio.xlsx", index=False)


  

In [33]:
# De Información: 

valores_unicos = df_tareas['PROYECTO'].nunique()

# Imprimir los valores únicos
print(valores_unicos)

28


In [30]:
proyectos_unicos = df_tareas["CODIGO"].nunique()
print(proyectos_unicos)

567
